<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg" />
</center> 
     
## <center>  [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 

#### <center> Author: [Yury Kashnitsky](https://yorko.github.io) (@yorko) 

# <center>Assignment #2. Fall 2019
## <center> Part 2. Gradient boosting

**In this assignment, you're asked to beat a baseline in the ["Flight delays" competition](https://www.kaggle.com/c/flight-delays-fall-2018).**

This time we decided to share a pretty decent CatBoost baseline, you'll have to improve the provided solution.

Prior to working on the assignment, you'd better check out the corresponding course material:
 1. [Classification, Decision Trees and k Nearest Neighbors](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic03_decision_trees_kNN/topic3_decision_trees_kNN.ipynb?flush_cache=true), the same as an interactive web-based [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-3-decision-trees-and-knn) 
 2. Ensembles:
  - [Bagging](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part1_bagging.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-1-bagging)
  - [Random Forest](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part2_random_forest.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-2-random-forest)
  - [Feature Importance](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part3_feature_importance.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-3-feature-importance)
 3. - [Gradient boosting](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic10_boosting/topic10_gradient_boosting.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-10-gradient-boosting) 
   - Logistic regression, Random Forest, and LightGBM in the "Kaggle Forest Cover Type Prediction" competition: [Kernel](https://www.kaggle.com/kashnitsky/topic-10-practice-with-logit-rf-and-lightgbm) 
 4. You can also practice with demo assignments, which are simpler and already shared with solutions:
  - "Decision trees with a toy task and the UCI Adult dataset": [assignment](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees) + [solution](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees-solution)
  - "Logistic Regression and Random Forest in the credit scoring problem": [assignment](https://www.kaggle.com/kashnitsky/assignment-5-logit-and-rf-for-credit-scoring) + [solution](https://www.kaggle.com/kashnitsky/a5-demo-logit-and-rf-for-credit-scoring-sol)
 5. There are also 7 video lectures on trees, forests, boosting and their applications: [mlcourse.ai/video](https://mlcourse.ai/video) 
 6. mlcourse.ai tutorials on [categorical feature encoding](https://www.kaggle.com/waydeherman/tutorial-categorical-encoding) (by Wayde Herman) and [CatBoost](https://www.kaggle.com/mitribunskiy/tutorial-catboost-overview) (by Mikhail Tribunskiy)
 7. Last but not the least: [Public Kernels](https://www.kaggle.com/c/flight-delays-fall-2018/notebooks) in this competition

### Your task is to:
 1. beat **"A2 baseline (10 credits)"** on Public LB (**0.75914** LB score)
 2. rename your [team](https://www.kaggle.com/c/flight-delays-fall-2018/team) in full accordance with A1 and the [course rating](https://docs.google.com/spreadsheets/d/15e1K0tg5ponA5R6YQkZfihrShTDLAKf5qeKaoVCiuhQ/) (to appear on 16.09.2019)
 
This task is intended to be relatively easy. Here you are not required to upload your reproducible solution.
 
### <center> Deadline for A2: 2019 October 6, 20:59 CET (London time)

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import warnings
import numpy as np
import pandas as pd
import catboost

from catboost import cv, Pool
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [194]:
days_from_start = {
    1: 0, 2: 31, 3: 59, 4: 90, 5: 120, 6: 151, 7: 181,
    8: 212, 9: 243, 10: 273, 11: 304, 12: 334
}

days_in_month = {
    1: 31,  2: 28,  3: 31,  4: 30, 5: 31, 6: 30,
    7: 31, 8: 31, 9: 30,  10: 31, 11: 30, 12: 31
}

drop = lambda data, names: data.drop(names, axis=1, inplace=True)

drop_names = "days_in_m days".split()

season_from_moth = {12: "winter", 1: "winter", 2: "winter",
                    3: "spring", 4: "spring", 5: "spring", 
                    6: "summer", 7: "summer", 8: "summer", 
                    9: "autumn", 10: "autumn", 11: "autumn"
                   }

weekend = {1: "workday", 2: "workday",  3: "workday", 4: "workday",
           5: "workday", 6: "weekend", 7: "weekend", }

In [195]:
def change_data(data):
    data["days_in_m"] = data.Month.map(days_in_month)
    
    data.DayofMonth = data.DayofMonth + data.DepTime // 2401
    data.DayOfWeek = data.DayOfWeek + data.DepTime // 2401
    data.Month = data.Month + data.DayofMonth // (data.days_in_m + 1)
    
    data.DepTime = data.DepTime % 2401
    data.DepTime = data.DepTime + (df.DepTime == 0)
    
    data.DepTime = data.DepTime  // 100 * 60 + data.DepTime % 100
    
    data.DayofMonth = data.DayofMonth % (data.days_in_m + 1)
    data.DayofMonth = data.DayofMonth + (data.DayofMonth == 0)
    
    data.DayOfWeek = data.DayOfWeek % 8
    data.DayOfWeek = data.DayOfWeek + (data.DayOfWeek == 0)
    
    data["season"] = data.Month.map(season_from_moth)
    # data.Distance = data.Distance.apply(np.log2)**2
    data["day_class"] = data.DayOfWeek.map(weekend)
    
    data["days"] = data.Month.map(days_from_start) + data.DayofMonth

    data['sin_time'] = np.sin(2 * np.pi * data.DepTime / (24*60))
    data['cos_time'] = np.cos(2 * np.pi * data.DepTime / (24*60))

    data["sin_days"] = np.sin(2 * np.pi * data.days / (365))
    data["cos_days"] = np.cos(2 * np.pi * data.days / (365))

    

In [157]:
names =  "Month DayofMonth DayOfWeek".split()

**Read the data**

In [196]:
df = pd.read_csv('../../data/flight_delays_train.csv')
test_df = pd.read_csv('../../data/flight_delays_test.csv')

df.dep_delayed_15min = df['dep_delayed_15min'].map({'Y': 1, 'N': 0})

for name in names:
    test_df[name] = test_df[name].str[2:].astype(int)
    df[name] = df[name].str[2:].astype(int)
        


In [197]:
change_data(df)
change_data(test_df)

for d in (df, test_df):
    drop(d, drop_names[:-1])


In [198]:
for name in names:
    test_df[name] = test_df[name].astype(str)
    df[name] = df[name].astype(str)   

In [139]:
for d in (df, test_df):
    drop(d, "Month DayofMonth DayOfWeek".split())
    

In [199]:
df['flight'] = df['Origin'] + ' --> ' + df['Dest']
test_df['flight'] = test_df['Origin'] + ' --> ' + test_df['Dest']

In [200]:
test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,season,day_class,days,sin_time,cos_time,sin_days,cos_days,flight
0,7,25,3,375,YV,MRY,PHX,598,summer,workday,206,0.997859,-0.065403,-0.393590,-0.919286,MRY --> PHX
1,4,17,2,459,WN,LAS,HOU,1235,spring,workday,107,0.908143,-0.418660,0.963471,-0.267814,LAS --> HOU
2,12,2,7,411,MQ,GSP,ORD,577,winter,weekend,336,0.975342,-0.220697,-0.478734,0.877960,GSP --> ORD
3,3,25,7,974,WN,BWI,MHT,377,spring,weekend,84,-0.894934,-0.446198,0.992222,0.124479,BWI --> MHT
4,6,6,3,905,UA,ORD,STL,258,summer,workday,157,-0.722364,-0.691513,0.425000,-0.905193,ORD --> STL


In [201]:
df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,season,day_class,days,sin_time,cos_time,sin_days,cos_days,flight
0,8,21,7,1174,AA,ATL,DFW,732,0,summer,weekend,233,-0.917060,0.398749,-0.763889,-0.645348,ATL --> DFW
1,4,20,3,948,US,PIT,MCO,834,0,spring,workday,110,-0.838671,-0.544639,0.948362,-0.317191,PIT --> MCO
2,9,2,5,862,XE,RDU,CLE,416,0,autumn,workday,245,-0.580703,-0.814116,-0.880012,-0.474951,RDU --> CLE
3,11,25,6,615,OO,DEN,MEM,872,0,autumn,weekend,329,0.442289,-0.896873,-0.580800,0.814046,DEN --> MEM
4,10,7,6,1108,WN,MDW,OMA,423,1,autumn,weekend,280,-0.992546,0.121869,-0.994218,0.107381,MDW --> OMA


---
---
---

In [202]:
categ_feat_idx = np.where(df.drop('dep_delayed_15min', axis=1).dtypes == 'object')[0]
print(categ_feat_idx)

X_train = df.drop('dep_delayed_15min', axis=1).values
y_train = df['dep_delayed_15min'].values
X_test = test_df.values

X_train_part, X_valid, \
y_train_part, y_valid = train_test_split(X_train, y_train,
                                         test_size=0.3,
                                         random_state=17,
                                         shuffle=True
                                         )
                                        

[ 0  1  2  4  5  6  8  9 15]


#### Train

In [203]:
ctb = CatBoostClassifier(random_seed=17, silent=True)

In [204]:
%%time
ctb.fit(X_train_part,
        y_train_part,
        cat_features=categ_feat_idx);

Wall time: 1min 29s


In [205]:
ctb_valid_pred = ctb.predict_proba(X_train_part)[:, 1]
print(roc_auc_score(y_train_part, ctb_valid_pred))

ctb_valid_pred = ctb.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, ctb_valid_pred))

0.8586773677347292
0.7611289678801213


In [206]:
ctb.get_feature_importance()

array([ 3.21001048,  4.45424279,  3.8586891 , 11.861239  ,  8.33813846,
        7.10064081,  6.61456091,  5.92556932,  1.56502971,  0.32700727,
        2.99743206, 18.94520507, 12.08132555,  3.93214189,  3.66570752,
        5.12306006])

0.7570888330382527 - *1000 только время
0.7635517184286487 - + fr0m to by cater

drop origin and dest,  sin, cos, from to - 0.7591306620174307  
casual sin cos time, back origin and dest, from to - 0.7635517184286487   
drop dest. 0.7604999899352793, (0.8718482770450207)  
add days - 0.7696358301705235 (0.7658038656143986)  
add season - 0.7607041024708254  
add weakday - 0.7617929928145148 / 0.8672219690617864  
first as numeric - 0.7482642455240555  
8 features - 0.7584589262375663
0.7611289678801213

K Fold

In [180]:
test_score = []
train_score = []
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

i = 0
for test_index, train_index in skf.split(X_train, y_train):
    
    X_train_1, X_test_1 = X_train[train_index, :], X_train[test_index, :]
    y_train_1, y_test_1 = y_train[train_index], y_train[test_index]
    
    
    _ctb = CatBoostClassifier(random_seed=17,
                              silent=True
                             ).fit(X_train_1, y_train_1, cat_features=categ_feat_idx)
    
    test_score.append(roc_auc_score(y_test_1,
                                    _ctb.predict_proba(X_test_1)[:, 1]
                                   )
                     )
    train_score.append(roc_auc_score(y_train_1, _ctb.predict_proba(X_train_1)[:, 1]))
    
    i += 1
    print(i)
    

1
2
3


In [181]:
print(np.mean(train_score))
print(train_score)

print()

print(np.mean(test_score))
print(test_score)

0.8123788727182845
[0.8144050900205388, 0.8061849163792483, 0.8165466117550662]

0.7428224723321925
[0.746539596900843, 0.7435141346766578, 0.7384136854190767]


0.7441712144561364
0.7439605741356733 - add season
season, day_class, drop month, dayofweek, dayofmonth - 0.733858795374717


params = \
{
# "iterations": 100,
"eval_metric": "AUC",
"verbose": False
#"devices": -1
}

pool = Pool(data=X_train,
            label=y_train,
            cat_features=categ_feat_idx)

scores = cv(pool,
            params=params,
            fold_count=5,
            plot="True",
            early_stopping_rounds=50)

----
----
----

Result

In [59]:
ctb = CatBoostClassifier(random_seed=17, silent=True)

In [60]:
%%time
ctb.fit(X_train, y_train,
        cat_features=categ_feat_idx);

Wall time: 1min 50s


In [94]:
ctb_test_pred = ctb.predict_proba(X_test)[:, 1]

In [63]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    sample_sub = pd.read_csv('../../data/sample_submission.csv', 
                             index_col='id')
    sample_sub['dep_delayed_15min'] = ctb_test_pred
    sample_sub.to_csv('ctb_pred_4.csv')

In [123]:
!head ctb_pred.csv

id,dep_delayed_15min
0,0.029813427154851885
1,0.045432589326845726
2,0.07735792880819185
3,0.26298352270959774
4,0.23812453664806577
5,0.1015030549268571
6,0.040331757718181545
7,0.2408800371330324
8,0.11304498960469873


In [127]:
!head ctb_pred_3.csv

id,dep_delayed_15min
0,0.025572166172376693
1,0.045981976846112715
2,0.07101399403966373
3,0.2905628227350732
4,0.2716783001872599
5,0.08292537325175686
6,0.04404007262086365
7,0.19666702216697723
8,0.07454022433182766


In [64]:
!head ctb_pred_4.csv

id,dep_delayed_15min
0,0.04204600812413802
1,0.04744355863289448
2,0.045877040873840425
3,0.2783059733111226
4,0.27047425085052357
5,0.09258748737744668
6,0.053454358999176735
7,0.17877175957797464
8,0.06612051141218417


In [49]:
!head ctb_pred_5.csv

id,dep_delayed_15min
0,0.037172779955821154
1,0.06319181761981149
2,0.04419190023283494
3,0.2804437695014472
4,0.2471133998367755
5,0.08769265924732995
6,0.04998421893664289
7,0.2136077081220574
8,0.10569904104193685
